In [2]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

C:\Users\posei\Anaconda3\envs\learn_ai\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [10]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file: 
        sound = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma: 
            stft = np.abs(librosa.stft(sound))
            
        result = np.array([])
        
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=sound, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate, ).T, axis=0)
            result=np.hstack((result, chroma))
        
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(sound, sr=sample_rate).T, axis=0)
            result = np.hstack((result, mel))
    return result

In [11]:
emotions = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}

observed = ['calm', 'happy', 'fearful', 'disgust']

In [12]:
def load_data(test_size=0.2):
    x, y = [], []
    
    for file in glob.glob("C:\\Users\\posei\\OneDrive\\Documents\\Soft Computing\\Speech Recognition\\dataset\\Actor_*\\*.wav"):
        filename = os.path.basename(file)
        emotion = emotions[filename.split("-")[2]]
        
        if emotion not in observed:
            continue
        
        feature = extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
        
    return train_test_split(np.array(x), y, test_size=test_size, random_state=0)

In [13]:
x_train, x_test, y_train, y_test = load_data(test_size=0.25)

C:\Users\posei\Anaconda3\envs\learn_ai\lib\site-packages\librosa\util\utils.py:826: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return (x > x_pad[inds1]) & (x >= x_pad[inds2])


In [14]:
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [18]:
print(f'features extracted: {x_train.shape[1]}')

features extracted: 180


In [20]:
model = MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [21]:
model.fit(x_train, y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [22]:
y_pred=model.predict(x_test)

In [23]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 70.83%
